In [1]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from numba import njit
from time import time_ns
import matplotlib.pyplot as plt

In [2]:
cmt_df = pd.read_csv('CMT.csv', index_col='Date')
cmt_df.index = pd.to_datetime(cmt_df.index)
cmt = cmt_df.values

In [3]:
cmt_df

,cmt0.25,cmt2,cmt3,cmt5,cmt7,cmt10
Date,,,,,,
1988-01-08,0.0675,0.0792,0.0816,0.0848,0.0880,0.0897
1988-01-15,0.0675,0.0761,0.0783,0.0813,0.0842,0.0860
1988-01-22,0.0670,0.0751,0.0773,0.0801,0.0829,0.0849
1988-01-29,0.0660,0.0722,0.0748,0.0776,0.0806,0.0826
1988-02-05,0.0640,0.0710,0.0732,0.0764,0.0795,0.0812
...,...,...,...,...,...,...
2000-06-02,0.0648,0.0652,0.0648,0.0637,0.0639,0.0615
2000-06-09,0.0644,0.0656,0.0650,0.0636,0.0636,0.0613
2000-06-16,0.0643,0.0638,0.0632,0.0619,0.0621,0.0599


In [4]:
@njit
def objective(se):
    T = 0.25
    A_T = np.exp((( (se[2]**2)/(2*se[1]**2) )-( se[0]/se[1] ))*T + (( se[0]/(se[1]**2) )-( (se[2]**2)/(se[1]**3) ))*(1-np.exp(-se[1]*T)) + ( (se[2]**2)/(4*se[1]**3) )*( (1-np.exp(-2*se[1]*T)) ))
    B_T = (1/se[1])*(1-np.exp(-se[1]*T))
    D_T = np.zeros(shape=(650,20))
    r0 = ((cmt[:,0]+np.log(A_T)/T)*(T/B_T)) 
    for i in np.arange(1,21):
        T = i/2
        A_T = np.exp((( (se[2]**2)/(2*se[1]**2) )-( se[0]/se[1] ))*T + (( se[0]/(se[1]**2) )-( (se[2]**2)/(se[1]**3) ))*(1-np.exp(-se[1]*T)) + ( (se[2]**2)/(4*se[1]**3) )*( (1-np.exp(-2*se[1]*T)) ))
        B_T = (1/se[1])*(1-np.exp(-se[1]*T))
        D_T[:,i-1] = A_T*np.exp(-B_T*r0)
    T = [2,3,5,7,10]
    Par_rates = np.zeros(shape=(650,5))
    for j,i in enumerate(T):
        Par_rates[:,j] = 0.02*(100-100*D_T[:,i*2-1])/np.sum(D_T[:,:i*2], axis=1)
    return np.sqrt(np.mean(np.square(Par_rates-cmt[:,1:])))

In [7]:
objective([0.02079224, 0.24311847, 0.03583904])

0.007025644676177797

In [6]:
a = [0.001]*3

b = (0,np.inf)
bnds = (b, b, b)
solution = minimize(objective,a,bounds=bnds)

In [7]:
solution

      fun: 0.007025644676164474
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([-1.15447582e-05,  6.76455420e-07,  3.76868675e-07])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 168
      nit: 31
     njev: 42
   status: 0
  success: True
        x: array([0.02079224, 0.24311847, 0.03583904])